<a href="https://colab.research.google.com/github/crystalloide/RAG/blob/main/LAB48_Fondamentaux_de_LangChain_et_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LAB48 : Découverte de LangChain et LangGraph

**Objectif :** Comprendre les différences entre les chaînes LangChain et les graphes LangGraph, et exécuter des exemples minimaux pour chacun.

**Durée estimée :** 15 à 20 minutes

**Livrable :** Un notebook exécutant les deux frameworks sur la même tâche et illustrant les différences structurelles.

## 1) Setup :

Installation des packages requis et configuration de la clé API.

In [1]:
# Installation des packages requiss
!pip install -q langchain langchain-openai langgraph openai python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 3.7 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

# Récupération de la clé API depuis les secrets Colab
# Pour ajouter : cliquez sur 🔑 dans le panneau de gauche
try:
    openai_api_key = userdata.get('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY'] = openai_api_key
    print("✓ Clé API OpenAI chargée depuis les secrets Colab")
except:
    print("⚠ Secrets Colab non configurés. Veuillez ajouter OPENAI_API_KEY.")
    print("Instructions : Cliquez sur 🔑 dans le panneau gauche > Ajouter un nouveau secret")

✓ Clé API OpenAI chargée depuis les secrets Colab


##2) LangChain : construction d'une une chaîne simple :
Idée centrale de LangChain : chaînes séquentielles d’appels LLM et de modèles de prompts.

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. Initialisation du modèle
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 2. Définition du template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a concise summarizer."),
    ("user", "Summarize this text in 3 bullet points:\n{text}")
])

# 3. Création de la chaîne avec LCEL (le pipe '|')
# On ajoute StrOutputParser() pour extraire directement le texte de la réponse
chain = prompt | llm | StrOutputParser()

# Texte d'exemple
# sample_text = """
# Agentic AI allows autonomous decision-making by combining reasoning, memory, and tools.
# It enables multi-agent collaboration and domain-specific applications.
# Challenges include safety, alignment, and evaluation.
# """

sample_text = """
L'IA agentive permet une prise de décision autonome en combinant raisonnement, mémoire et outils.
Elle permet la collaboration multi-agents et les applications spécifiques à un domaine.
Les défis incluent la sécurité, l'alignement et l'évaluation.
"""


# 4. Exécution (on utilise .invoke au lieu de .run)
print("\n--- LangChain Output ---\n")
result = chain.invoke({"text": sample_text})

print(result)


--- LangChain Output ---

- L'IA agentive facilite la prise de décision autonome grâce à l'intégration du raisonnement, de la mémoire et d'outils.
- Elle favorise la collaboration entre plusieurs agents et des applications spécialisées dans divers domaines.
- Les principaux défis à relever sont la sécurité, l'alignement et l'évaluation des systèmes.


### 👉 Point clé à retenir

LangChain facilite l'intégration des LLM, des modèles (templates) et des outils (tools) dans un pipeline linéaire.

## 3) LangGraph : construction d'un graphe simple :

Idée fondamentale de LangGraph :
- graphes orientés
- avec état (stateful)
- composés de nœuds (Directid Graphs)
- avec boucles
- avec banchements
- avec mémoire

In [10]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

# Define state
class State(TypedDict):
    text: str
    summary: str

# Define nodes
def summarize(state: State):
    response = llm.invoke(f"Summarize this text in 2 sentences:\n{state['text']}")
    return {"summary": response.content}

def reflect(state: State):
    response = llm.invoke(f"Critique this summary: {state['summary']}")
    return {"summary": state['summary'] + "\nCritique: " + response.content}

# Build graph
workflow = StateGraph(State)
workflow.add_node("summarize", summarize)
workflow.add_node("reflect", reflect)
workflow.set_entry_point("summarize")
workflow.add_edge("summarize", "reflect")
workflow.add_edge("reflect", END)
app = workflow.compile()

# Run
print("\n--- LangGraph Output ---\n")
result = app.invoke({"text": sample_text})
print(result["summary"])


--- LangGraph Output ---

L'IA agentive facilite la prise de décision autonome en intégrant raisonnement, mémoire et outils, tout en favorisant la collaboration entre plusieurs agents et des applications spécialisées. Cependant, elle pose des défis en matière de sécurité, d'alignement et d'évaluation.
Critique: Cette synthèse présente de manière concise les avantages et les défis associés à l'IA agentive. Voici quelques points de critique :

1. **Clarté et précision** : Le terme "IA agentive" pourrait être mieux défini pour les lecteurs qui ne sont pas familiers avec le concept. Une brève explication de ce que cela implique renforcerait la compréhension.

2. **Équilibre** : Bien que les avantages soient mentionnés, la formulation pourrait donner l'impression que les défis sont secondaires. Il serait bénéfique d'accorder plus de poids aux défis, en expliquant brièvement chacun d'eux (sécurité, alignement, évaluation) pour souligner leur importance.

3. **Exemples concrets** : L'ajout d

Extrait de code :
- Vous pouvez utiliser **Mermaid** (langage de script utilisé pour générer des diagrammes à partir de texte) pour définir le sens de lecture du graphique.
- **Mermaid** : https://mermaid.ai/app/projects

- **graph LR** signifie "graphique de gauche à droite" :

```
graph LR
  START((START)) --> summarize[summarize]
  summarize --> reflect[reflect]
  reflect --> END((END))
  
  subgraph State
    text
    summary
  end
```

### **Analyse des composants** :
Voici comment les données circulent à travers ton graphe :

- L'État (**State**) : C'est la mémoire partagée. Au début, on fournit le texte **text**. Après le premier nœud, l'état contient à la fois **text** et **summary**.

- Point d'entrée **START**: Le graphe commence toujours par le nœud défini dans set_entry_point, ici summarize.

- Nœud **summarize** : Prend le texte brut, génère un résumé via l'LLM, et met à jour la clé summary dans l'état.

- Nœud **reflect** : Récupère le summary créé à l'étape précédente, demande une critique à l'LLM, et l'ajoute à la suite du résumé existant.

- Fin (**END**) : Une fois que reflect a terminé son exécution, le graphe atteint le marqueur spécial END et s'arrête.

**Note** :
- Dans ce schéma, l'état est "persistant" entre les nœuds.
- Chaque fonction ne renvoie que la modification qu'elle souhaite apporter
- et LangGraph s'occupe de fusionner cela dans l'objet State global.

### 👉 Point clé à retenir

LangGraph est conçu pour la logique de branchement et les boucles de rétroaction :
- il est plus flexible que les chaînes séquentielles de LangChain.

## 4) Comparaison de LangChain vs LangGraph :

| Aspect | LangChain | LangGraph |
|--------|-----------|----------|
| **Abstraction** | Chains (linear pipelines) | Graphs (nodes + edges + state) |
| **Avantages** | Simple, fast to prototype | Complex workflows, feedback loops |
| **Usages** | Small apps, demos, simple RAG | Agent orchestration, multi-step reasoning |
| **Mémoire** | Ad-hoc (ConversationBuffer) | Built-in state objects |
| **Design style** | Prompt-template centric | State-machine centric |

## 5) Mini-projet (optionnel)

Essayez les exercices suivants :

1. Modifiez la chaîne LangChain pour inclure une 2nde étape (par exemple, un rewriter de style).

2. Modifiez le graphe LangGraph pour ajouter une boucle : après chaque critique, relancez `summarize` jusqu’à ce que la critique soit « satisfied ».

3. Comparez les résultats.


### Exercice n°1 : LangChain avec une chaîne à 2 étapes :

Remarque :
- Aujourd'hui, LangChain recommande d'utiliser le "Pipe" (|).
- C'est beaucoup plus lisible, plus rapide et plus facile à déboguer.

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Configuration des prompts
# summarize_prompt = ChatPromptTemplate.from_template("Summarize this text in 3 bullet points:\n{text}")
# rewrite_prompt = ChatPromptTemplate.from_template("Rewrite this summary in a formal, academic style:\n{summary}")
summarize_prompt = ChatPromptTemplate.from_template("Résume ce texte en 3 points clés :\n{text}")
rewrite_prompt = ChatPromptTemplate.from_template("Réécrit ce résumé dans un style formel et académique ::\n{summary}")

# Création de la chaîne moderne LCEL (LangChain Expression Language)
# On utilise un dictionnaire pour transmettre le résultat de l'étape 1 à l'étape 2
chain = (
    {"summary": summarize_prompt | llm | StrOutputParser()}
    | RunnablePassthrough.assign(
        formal_summary = lambda x: (rewrite_prompt | llm | StrOutputParser()).invoke({"summary": x["summary"]})
    )
)

# Exécution
result = chain.invoke({"text": sample_text})

print(f"Résumé:\n{result['summary']}")
print(f"\nRéécriture en style académique formel:\n{result['formal_summary']}")

Résumé:
1. L'IA agentive facilite la prise de décision autonome en intégrant raisonnement, mémoire et outils.
2. Elle favorise la collaboration entre plusieurs agents et permet des applications spécialisées dans divers domaines.
3. Les principaux défis à relever concernent la sécurité, l'alignement des objectifs et l'évaluation des performances.

Réécriture en style académique formel:
L'intelligence artificielle agentive constitue un outil facilitant la prise de décision autonome, en intégrant des mécanismes de raisonnement, de mémoire et d'outils adaptés. Elle encourage également la collaboration entre plusieurs agents, permettant ainsi le développement d'applications spécialisées dans divers domaines d'activité. Toutefois, les principaux défis à surmonter dans ce domaine concernent la sécurité, l'alignement des objectifs des agents et l'évaluation de leurs performances.


### Exercice n°2 : LangGraph avec une boucle de Feedback :

In [25]:
# TODO: Add a loop that re-summarizes until the critique is positive

from typing import Literal

class LoopState(TypedDict):
    text: str
    summary: str
    critique: str
    iteration: int

def summarize_v2(state: LoopState):
    iteration = state.get('iteration', 0) + 1
    response = llm.invoke(f"Summarize this text in 2 sentences:\n{state['text']}")
    return {"summary": response.content, "iteration": iteration}

def critique(state: LoopState):
    response = llm.invoke(
        #f"Rate this summary from 1-10 and explain why. Summary: {state['summary']}\n"
        #f"Return ONLY the number followed by your explanation."
        f"Évalue ce résumé de 1 à 10 et explique pourquoi. Résumé: {state['summary']}\n"
        f"Renvoie UNIQUEMENT le nombre suivi de ton explication."
    )
    return {"critique": response.content}

def should_continue(state: LoopState) -> Literal["summarize", "end"]:
    """Check si le score "critique" est >= 8 ou si on a fait 3 itérations"""
    critique_text = state.get('critique', '')
    iteration = state.get('iteration', 0)

    # Extract score (first number in critique)
    import re
    match = re.search(r'\d+', critique_text)
    score = int(match.group()) if match else 0

    if score >= 9 or iteration >= 3:
        return "end"
    return "summarize"

# Construction du graph avec une boucle :
workflow_loop = StateGraph(LoopState)
workflow_loop.add_node("summarize", summarize_v2)
workflow_loop.add_node("critique", critique)
workflow_loop.set_entry_point("summarize")
workflow_loop.add_edge("summarize", "critique")
workflow_loop.add_conditional_edges(
    "critique",
    should_continue,
    {
        "summarize": "summarize",
        "end": END
    }
)
app_loop = workflow_loop.compile()

# Exécution : on fait 3 itérations maximum ou à partir d'une note de 9+
print("\n--- LangGraph avec boucle de Feedback ---\n")
result = app_loop.invoke({"text": sample_text, "iteration": 0})
print(f"Iterations: {result['iteration']}")
print(f"\nRésumé final:\n-{result['summary']}")
print(f"\nCritique finale:\n- Note : {result['critique']}")


--- LangGraph avec boucle de Feedback ---

Iterations: 3

Résumé final:
-L'IA agentive facilite la prise de décision autonome en intégrant raisonnement, mémoire et outils, tout en favorisant la collaboration entre plusieurs agents et des applications spécialisées. Cependant, elle pose des défis en matière de sécurité, d'alignement et d'évaluation.

Critique finale:
- Note : 8

Ce résumé est concis et couvre les points essentiels concernant l'IA agentive, notamment ses avantages en matière de prise de décision autonome et de collaboration, ainsi que les défis qu'elle pose. Cependant, il pourrait être amélioré en fournissant des exemples concrets ou en développant davantage les défis mentionnés pour donner une meilleure compréhension du sujet.


## ✅ Conclusion et synthèse:

En terminant ce TP, vous savez désormais comment :

- Créer une chaîne LangChain de base

- Créer un workflow LangGraph avec gestion d'état

- Choisir le framework le plus adapté à chaque situation

- Construire des chaînes à plusieurs étapes et des boucles de rétroaction

### Principales différences :

- **LangChain** : Idéal pour les workflows simples et linéaires et le prototypage rapide

- **LangGraph** : Idéal pour les workflows complexes avec embranchements, boucles et gestion d'état

## Ressources supplémentaires pour aller plus loin :

- [Documentation de LangChain](https://python.langchain.com/)
- [Documentation de LangGraph](https://langchain-ai.github.io/langgraph/)
- [Référence de l'API OpenAI](https://platform.openai.com/docs/api-reference)